In [66]:
# to ensure that the modules can be imported, as they are located in a different folder, add the package root to the path:

import sys
sys.path.insert(0, "../")


In [67]:
import librosa
from sktalk import read_audio
import IPython.display as ipd

In [86]:
import subprocess
import json
import numpy as np


def load_audio(file_path):
    cmd = [
        "ffprobe",
        "-v", "quiet",
        "-print_format", "json",
        "-show_streams",
        file_path
    ]

    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    output = json.loads(result.stdout)

    sample_rate = None
    for stream in output["streams"]:
        if stream["codec_type"] == "audio":
            sample_rate = stream["sample_rate"]
            no_channels = str(stream["channels"])

    if sample_rate is None:
        raise ValueError("No audio stream found in the file")

    cmd = ["ffmpeg", "-i", file_path, '-f', 's16le',
           '-acodec', 'pcm_s16le',
           '-ar', sample_rate,
           '-ac', no_channels,
           '-']

    print((' ').join(cmd))
    pipe = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    raw_audio = pipe.stdout
    audio_array = np.frombuffer(raw_audio, dtype="int16")
    audio_array = audio_array.astype(np.float32) / np.iinfo(np.int16).max

    return audio_array, int(sample_rate)



In [87]:
wavaddress = "../data/catalan_demo.wav"

d_ffmpeg, r_ffmpeg = load_audio(wavaddress)
d_librosa, r_librosa = librosa.load(wavaddress)

print(r_librosa, r_ffmpeg)

ffmpeg -i ../data/catalan_demo.wav -f s16le -acodec pcm_s16le -ar 16000 -ac 2 -
22050 16000


In [20]:
ipd.Audio(wavaddress) # load a local WAV file

In [88]:
ipd.Audio(d_ffmpeg, rate=r_ffmpeg)

In [26]:
import soundfile as sf

sf.write('../data/output.wav', d_librosa, r_librosa, subtype='PCM_24')